In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import transforms

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=transforms.ToTensor(),
)
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=transforms.ToTensor(),
)

100%|██████████| 26.4M/26.4M [00:11<00:00, 2.23MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 484kB/s]
100%|██████████| 4.42M/4.42M [00:02<00:00, 1.62MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 5.32MB/s]


In [3]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for x, y in test_dataloader:
    print(x.shape, y.shape)

torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size

In [4]:
device = torch.accelerator.current_accelerator().type if torch.cuda.is_available() else "cpu"
device

'cpu'

In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relue_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relue_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relue_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [10]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()

    for batch, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)

        pred = model(x)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(x)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [11]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            test_loss =+ loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [12]:
epoch = 5
for t in range(epoch):
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

loss: 1.325868 [   64/60000]
loss: 1.310236 [ 6464/60000]
loss: 1.147833 [12864/60000]
loss: 1.245177 [19264/60000]
loss: 1.128601 [25664/60000]
loss: 1.152404 [32064/60000]
loss: 1.157392 [38464/60000]
loss: 1.109993 [44864/60000]
loss: 1.150952 [51264/60000]
loss: 1.062401 [57664/60000]
Test Error: 
 Accuracy: 65.2%, Avg loss: 0.006660 

loss: 1.152818 [   64/60000]
loss: 1.158597 [ 6464/60000]
loss: 0.978037 [12864/60000]
loss: 1.107359 [19264/60000]
loss: 0.984972 [25664/60000]
loss: 1.016026 [32064/60000]
loss: 1.036895 [38464/60000]
loss: 0.993969 [44864/60000]
loss: 1.035493 [51264/60000]
loss: 0.961087 [57664/60000]
Test Error: 
 Accuracy: 66.1%, Avg loss: 0.005759 

loss: 1.032751 [   64/60000]
loss: 1.060093 [ 6464/60000]
loss: 0.861360 [12864/60000]
loss: 1.014574 [19264/60000]
loss: 0.894978 [25664/60000]
loss: 0.920321 [32064/60000]
loss: 0.957652 [38464/60000]
loss: 0.918054 [44864/60000]
loss: 0.955463 [51264/60000]
loss: 0.893667 [57664/60000]
Test Error: 
 Accuracy: 67

In [13]:
# https://docs.pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html#saving-models